In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
#import modules
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM,Embedding,Dropout, add

In [14]:
#LOAD VGG16
model=VGG16()
model=Model(inputs=model.inputs, outputs=model.layers[-2].output)
print(model.summary())

553467096/553467096 [==============================] - 4s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                          

In [15]:
##extraction

features={}
directory='/kaggle/input/muzzle-pics/muzzle_only'

for img_name in tqdm(os.listdir(directory)):
    img_path= directory+'/'+ img_name
    img=load_img(img_path,target_size=(224,224))
    img=img_to_array(img)
    img=img.reshape(1,img.shape[0],img.shape[1],img.shape[2])
    img=preprocess_input(img)
    feature=model.predict(img,verbose=0)
    image_id=img_name.split('.')[0]
    #store features
    features[image_id]=feature

  0%|          | 0/31 [00:00<?, ?it/s]

In [17]:
WORKING_DIR='/kaggle/working/'
#storing features in pickle
pickle.dump(features,open(os.path.join(WORKING_DIR,'features.pkl'),'wb'))

In [21]:
from sklearn.metrics.pairwise import cosine_similarity  # You can use cosine similarity for feature comparison

# Load the pre-extracted features from the pickle file
WORKING_DIR = '/kaggle/working/'
features = pickle.load(open(os.path.join(WORKING_DIR, 'features.pkl'), 'rb'))

# Function to extract features from a new image
def extract_features_from_new_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
    img = preprocess_input(img)
    feature = model.predict(img, verbose=0)
    return feature

# Path to the new image you want to compare
new_image_path = '/kaggle/input/test-imgs/test dataset/C.png'

# Extract features from the new image
new_image_features = extract_features_from_new_image(new_image_path)

# Initialize variables to track the best match and similarity
#best_match = None
best_similarity = 0.0

# Compare the new image features with features in the dataset
for image_id, dataset_feature in features.items():
    # Calculate the cosine similarity between the new image features and dataset features
    similarity = cosine_similarity(new_image_features, dataset_feature)
    
    if similarity > best_similarity:
        best_similarity = similarity
        #best_match = image_id

# Set a similarity threshold (you can adjust this)
similarity_threshold = 0.9

if best_similarity > similarity_threshold:
    print("match found.")
else:
    print("No match found.")



No match found.
